<span STYLE="font-size:150%"> 
    Segment microCT scans
</span>

Docker image: gnasello/slicer-env:2023-07-06 \
Latest update: 10 March 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Install libraries

Run the command below one time only, then comment it

In [ ]:
# !git clone https://github.com/gabnasello/pyslicer.git
# !PythonSlicer -m pip install -e pyslicer/
# !PythonSlicer -m pip install matplotlib

# import os
# os._exit(00)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
volume_file = 'microCT_volume/microCT_volume_preview.nrrd'

path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Make ```segmented_volumes``` folder

In [ ]:
output_directory = Path('segmented_volumes')

output_directory.mkdir(parents=True, exist_ok=True)

# Thresholding

## Create segmentation-related nodes

Create segmentation node

In [ ]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Check different automatic segmentation algorithms

The first time you analyze an image/channel. Go to the ```Segment Editor```, add a new segment and choose the thresholding method. There you test different automatic thresholding technique and choose the one working best. For later analysis, you can just copy the thresholding values in the ```segments_greyvalues``` object at the beginning of this notebook.

In [ ]:
ps.volume.plot_histogram(masterVolumeNode, yscale='log')

Get automatic thresholding values, as indicated in [this script](https://github.com/jzeyl/3D-Slicer-Scripts/blob/db51967cc642837e8bae0fea1585a95074d8420b/load_dicom_modified_otsu.py#L56)

In [ ]:
methods = [
            'HUANG',
            'INTERMODES',
            'ISO_DATA',
            'KITTLER_ILLINGWORTH',
            'LI',
            'MAXIMUM_ENTROPY',
            'MOMENTS',
            'OTSU',
            'RENYI_ENTROPY',
            'SHANBHAG',
            'TRIANGLE',
            'YEN'
            ]

thresholds = dict.fromkeys(methods, None)
thresholds

Otsu thresholding

In [ ]:
method = 'OTSU'

threshold = ps.segmentation.compute_threshold(method = method, volumeNode = masterVolumeNode)

thresholds[method.upper()] = threshold

print(method + " threshold: " + str(threshold))
ps.volume.plot_histogram(masterVolumeNode, threshold = threshold, title = method, yscale='log')

IsoData thresholding

In [ ]:
method = 'ISO_DATA'

threshold = ps.segmentation.compute_threshold(method = method, volumeNode = masterVolumeNode)

thresholds[method.upper()] = threshold

print(method + " threshold: " + str(threshold))
ps.volume.plot_histogram(masterVolumeNode, threshold = threshold, title = method, yscale='log')

MOMENTS thresholding

In [ ]:
method = 'MOMENTS'

threshold = ps.segmentation.compute_threshold(method = method, volumeNode = masterVolumeNode)

thresholds[method.upper()] = threshold

print(method + " threshold: " + str(threshold))
ps.volume.plot_histogram(masterVolumeNode, threshold = threshold, title = method, yscale='log')

MAXIMUM_ENTROPY thresholding

INTERMODES thresholding

## Select thresholding values

In [ ]:
thresholds

In [ ]:
segments_greyvalues = {
    "Bone": [thresholds['MOMENTS']*0.9, thresholds['ISO_DATA']], # [MaxEntropy, MaxIntensity]
    }

segments_greyvalues

## Create segments by thresholding

In [ ]:
# Avoid overwrite of overlapping segments
segmentEditorNode.SetOverwriteMode(slicer.vtkMRMLSegmentEditorNode.OverwriteNone)

ps.segmentation.segments_by_thresholding(segments_greyvalues, 
                                         segmentationNode,
                                         segmentEditorNode,
                                         segmentEditorWidget)

Set segments color

In [ ]:
segments_color = {
    "Bone": (0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
    }

segments_color

In [ ]:
ps.segmentation.set_segments_color(segments_color, segmentationNode)

Get closed surface representation of the segment and visualize segment in 3D view, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

<span style="color:red; font-size:200%">Manual Task</span>
# Draw Region of Interest in Slicer UI

Before running the code below, go to the Slicer UI and define a **ROI** from the ```markups``` module.

Set the name of the ROI node to ```R```. Any name is actually fine, but you should change the line below accoridngly.

In [ ]:
nodename = 'R'
roiNode = slicer.util.getNode(nodename)

In [ ]:
f_output = 'R.mrk.json'

slicer.util.exportNode(roiNode, output_directory / f_output)

# Create segment with ROI

## Crop original volume based on a Region of Interest (ROI)

In [ ]:
# Define a function to crop a volume based on an ROI node and output to a specified node
def crop_volume_based_on_roi(volumeNode, roiNode, outputVolumeName="CroppedVolume"):
    # Create and name the output volume node
    outputVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", outputVolumeName)
    
    # Create a Crop Volume parameter node
    cropVolumeParameters = slicer.vtkMRMLCropVolumeParametersNode()
    cropVolumeParameters.SetInputVolumeNodeID(volumeNode.GetID())
    cropVolumeParameters.SetROINodeID(roiNode.GetID())
    cropVolumeParameters.SetOutputVolumeNodeID(outputVolumeNode.GetID())
    
    # Optionally set isotropic resampling (if desired)
    cropVolumeParameters.SetIsotropicResampling(False)  # Set to True if isotropic spacing is needed

    # Add the crop volume parameters node to the MRML scene
    slicer.mrmlScene.AddNode(cropVolumeParameters)
    
    # Run the crop volume logic
    cropVolumeLogic = slicer.modules.cropvolume.logic()
    cropVolumeLogic.Apply(cropVolumeParameters)
    
    return outputVolumeNode


In [ ]:
# Crop the volume based on the ROI
croppedVolume = crop_volume_based_on_roi(masterVolumeNode, roiNode, outputVolumeName="croppedVolume")

## Create segmentation node within the cropped volume node

Create segmentation node

In [ ]:
segmentationNode_croppedVolume = ps.segmentation.segmentationNode(name='Segmentation_croppedVolume')

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget_croppedVolume, segmentEditorNode_croppedVolume = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode_croppedVolume, 
                                                                                                         masterVolumeNode = croppedVolume)

## Create segment covering all cropped volume node

In [ ]:
segments_greyvalues = {
    "ROI": [0, 2**16], # [MaxEntropy, MaxIntensity]
    }

segments_greyvalues

In [ ]:
ps.segmentation.segments_by_thresholding(segments_greyvalues, 
                                         segmentationNode_croppedVolume,
                                         segmentEditorNode_croppedVolume,
                                         segmentEditorWidget_croppedVolume)

## Export ROI Mask segmentation as `seg.nrrd` file (labelmap node)

Export a selection of segments (identified by their names), from [slicer tutorial](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-labelmap-node-from-segmentation-node)

In [ ]:
seg_name = "ROI"

In [ ]:
mask_labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = seg_name,
                                                                   segmentationNode = segmentationNode_croppedVolume,
                                                                   volumeNode = croppedVolume)

In [ ]:
filename_output = seg_name + '.seg.nrrd'

slicer.util.exportNode(mask_labelmap, output_directory / filename_output)

# Visualize segmentations in 3D view

Get viewNode

In [ ]:
view = slicer.app.layoutManager().threeDWidget(0).threeDView()
viewNode = view.mrmlViewNode()

# Switch off cube and labels
viewNode.SetAxisLabelsVisible(False)
viewNode.SetBoxVisible(False)

color = (28/255, 29/255, 36/255)
# Set view background to RGB color of choice
viewNode.SetBackgroundColor(color[0], color[1], color[2])
viewNode.SetBackgroundColor2(color[0], color[1], color[2])

# Se?t Orthographic rendering, which is required to show the ruler in a 3D view
viewNode.SetRenderMode(viewNode.Orthographic)

# Set thick and white ruler
viewNode.SetRulerType(2) # 2 - thick
viewNode.SetRulerColor(0) # 0 - white

# Get camera position
cameraNode = slicer.modules.cameras.logic().GetViewActiveCameraNode(viewNode)

## Get camera position in 3D view

In [ ]:
position = cameraNode.GetPosition()

viewAngle = cameraNode.GetViewAngle()

viewUp = cameraNode.GetViewUp()

focalPoint = cameraNode.GetFocalPoint()

parallelScale = cameraNode.GetParallelScale()

In [ ]:
df1 = pd.DataFrame({'position':position,
                    'viewUp':viewUp,
                    'focalPoint':focalPoint})

df2 = pd.DataFrame({'viewAngle':[viewAngle],
                    'parallelScale':[parallelScale]})

df_camera = pd.concat([df1,df2], axis=1)
df_camera

In [ ]:
outputfile = output_directory / 'camera_view.csv'
df_camera.to_csv(outputfile, index=False, na_rep='NULL')